In [49]:
from pathlib import Path
import os
from PIL import Image

#ORIGINAL_THUMBNAIL_FOLDER_PATH = Path(".\\train_thumbnails")
PROCESSED_THUMBNAIL_FOLDER_PATH = Path("/Users/ryan/Downloads/TF_data/processed_train_thumbnails")

RESIZE_WIDTH = 64
RESIZE_HEIGHT = 64

# Resize all images to the same dimensions

for thumbnail_path in PROCESSED_THUMBNAIL_FOLDER_PATH.iterdir():
    img = Image.open(os.path.join(".", thumbnail_path))

    new_img = img.resize((RESIZE_WIDTH, RESIZE_HEIGHT))

    new_img.save(os.path.join(PROCESSED_THUMBNAIL_FOLDER_PATH, os.path.basename(thumbnail_path)))


In [72]:
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torchvision.transforms as transforms
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import csv

import numpy as np
import os.path
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import ConcatDataset


label_to_num_dict = {
    "HGSC": 0,
    "LGSC": 1,
    "EC": 2,
    "CC": 3,
    "MC": 4
}

image_id_list = []
label_list = []

with open("train.csv", newline="") as csvfile:
  fileReader = csv.DictReader(csvfile, delimiter=",")
  for row in fileReader:
    if row["is_tma"] == "False":
      image_id_list.append(row["image_id"])
      label_list.append(label_to_num_dict[row["label"]])
#HGSC, LGSC, E, C, M

class_weights = []
for i in range(5):
  class_weights.append(len(label_list)/(label_list.count(i)*5))
class_weights = torch.FloatTensor(class_weights)
class ImageDataset(Dataset):
  def __init__(self, image_id_list, label_list, train_index):
    self.x_data = []
    self.y_data = []

    transform = A.Compose([A.ToFloat(), ToTensorV2()])

    for i in train_index:
      img = np.array(Image.open(os.path.join("processed_train_thumbnails", f"{image_id_list[i]}_thumbnail.png")))
      self.x_data.append(transform(image=img))
      self.y_data.append(label_list[i])


    self.len = len(self.x_data)

  def __getitem__(self, index):
    return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.len

class RotateImageDataset(Dataset):
    def __init__(self, image_id_list, label_list, train_index):
        self.x_data = []
        self.y_data = []

        transform = A.Compose([
            A.Rotate(limit=30),
            A.ToFloat(),
            ToTensorV2()
        ])

        for i in train_index:
            img = np.array(Image.open(os.path.join("processed_train_thumbnails", f"{image_id_list[i]}_thumbnail.png")))
            augmented_img = transform(image=img)['image']
            self.x_data.append(augmented_img)
            self.y_data.append(label_list[i])

        self.len = len(self.x_data)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len
class VFlippedImageDataset(Dataset):
    def __init__(self, image_id_list, label_list, train_index):
        self.x_data = []
        self.y_data = []

        transform = A.Compose([
            A.ToFloat(),
            A.VerticalFlip(p=0.9),  # Add horizontal flip with 50% probability
            ToTensorV2()
        ])

        for i in train_index:
            img = np.array(Image.open(os.path.join("processed_train_thumbnails", f"{image_id_list[i]}_thumbnail.png")))
            augmented_img = transform(image=img)['image']
            self.x_data.append(augmented_img)
            self.y_data.append(label_list[i])

        self.len = len(self.x_data)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [73]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.convlayer1 = nn.Conv2d(3, 32, kernel_size=(5, 5), stride=1, padding=1)
        self.dropout = nn.Dropout(0.25)
        self.maxpool = nn.MaxPool2d(kernel_size=(2, 2))
        self.convlayer2 = nn.Conv2d(32, 32, kernel_size=(5, 5), stride=1, padding=1)
        self.layer1 = nn.Linear(28800, 512)
        self.fc = nn.Linear(512, 5)

    def forward(self, X):
        out = self.convlayer1(X)
        out = nn.functional.relu(out)
        out = self.dropout(out)
        out = self.convlayer2(out)
        out = nn.functional.relu(out)
        out = self.maxpool(out)
        out = torch.flatten(out, start_dim=1)
        out = self.layer1(out)
        out = self.fc(out)
        return out


def reset_weights(m):
  for i in m.children():
    if hasattr(i, 'reset_parameters'):
      i.reset_parameters()
model = CNNModel()

criterion = nn.CrossEntropyLoss(weight = class_weights)

learning_rate = 0.0001
num_epochs = 15
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


num_folds = 15
kFold = StratifiedKFold(n_splits=num_folds, shuffle=True)
splits = kFold.split(image_id_list, label_list)
num_epochs = 15

results = {}
scheduler = ReduceLROnPlateau(optimizer, factor=0.1, patience=5, min_lr=1e-6)



In [74]:
for n,(train_index,test_index) in enumerate(splits):
  dataset_train = ImageDataset(image_id_list, label_list, train_index)
  dataset_train_hflipped = RotateImageDataset(image_id_list, label_list, train_index)
  dataset_train_vflipped = VFlippedImageDataset(image_id_list, label_list, train_index)

  dataset_test = ImageDataset(image_id_list, label_list, test_index)

  train_loader = DataLoader(dataset=dataset_train, batch_size=32, shuffle=True)
  hflipped_train_loader = DataLoader(dataset=dataset_train_hflipped, batch_size=32, shuffle=True)
  vflipped_train_loader = DataLoader(dataset=dataset_train_vflipped, batch_size=32, shuffle=True)

  test_loader = DataLoader(dataset=dataset_test, batch_size=32, shuffle=True)

  #model = CNNModel()
  model.apply(reset_weights)

  total_step = len(train_loader)

  for epoch in range(num_epochs):
    current_loss = 0.0

    for i, data in enumerate(train_loader):
      x_imgs, labels = data
      optimizer.zero_grad()

      # Forward pass
      output = model(x_imgs["image"])
      loss = criterion(output, labels)

      # Backward and optimize
      loss.backward()
      optimizer.step()
      scheduler.step(current_loss)

      current_loss += loss.item()
      #_, predicted = torch.max(output.data, 1)
      #print('Predictions for batch {}: {}'.format(i + 1, predicted))
      if (i+1) % 10 == 0:
        print ('Fold # {}, Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                .format(n, epoch+1, num_epochs, i+1, total_step, loss.item()))
    
    
    


  correct, all = 0,0
  with torch.no_grad():
    for i, data in enumerate(test_loader):
      x_imgs, labels = data

      output = model(x_imgs["image"])

      _, predicted  = torch.max(output.data, 1)
      print("LABELS: ", labels)
      print('PREDICTED :', predicted)
      all += labels.size(0)
      correct += (predicted == labels).sum().item()

    # Print accuracy
    print('Accuracy for fold %d: %d %%' % (n, 100.0 * correct / all))
    print('--------------------------------')
    results[n] = 100.0 * (correct / all)

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {num_folds} FOLDS')
print('--------------------------------')
sum = 0.0
for key, value in results.items():
  print(f'Fold {key}: {value} %')
  sum += value
print(f'Average: {sum/len(results.items())} %')

Fold # 0, Epoch [1/15], Step [10/15], Loss: 2.1792


IndexError: too many indices for tensor of dimension 4